In [2]:
import pandas as pd
import numpy as np

# Import BigQuery library
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

# Query the data

In [3]:
query = """
SELECT confidence_score, website_url_binary, twitter_binary, linkedin_binary, location_identifiers_value_binary, phone_number_binary, short_description_binary, legal_name_binary, identifier_permalink_binary, contact_email_binary, status_encoded, operating_status_encoded, revenue_range_encoded, rank_band, num_employees_encoded, ipo_status_encoded, years_since_founded_band, company_type_encoded, days_since_updated, has_category_groups
FROM `sandbox-b1.safe_prod_exploration.crunchbase_label_final`

"""
query_job = client.query(query)

model_df = query_job.to_dataframe()

model_df


,confidence_score,website_url_binary,twitter_binary,linkedin_binary,location_identifiers_value_binary,phone_number_binary,short_description_binary,legal_name_binary,identifier_permalink_binary,contact_email_binary,status_encoded,operating_status_encoded,revenue_range_encoded,rank_band,num_employees_encoded,ipo_status_encoded,years_since_founded_band,company_type_encoded,days_since_updated,has_category_groups
0,92,1,1,1,1,1,1,1,1,1,1,1,3,3,3,0,8,2,335,1
1,84,1,0,1,1,1,1,1,1,1,2,1,2,1,1,0,6,2,943,1
2,97,1,0,0,1,1,1,1,1,1,2,1,0,1,1,0,12,2,1392,1
3,50,0,0,0,1,0,1,1,1,0,2,1,0,1,0,0,0,0,1456,1
4,66,1,0,1,1,1,1,1,1,1,2,1,2,2,1,0,5,2,1007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231386,88,1,0,0,1,1,1,1,1,1,2,1,0,1,1,0,7,2,280,1
231387,85,1,1,1,1,1,1,1,1,0,2,1,1,1,1,0,0,2,951,1
231388,100,1,0,0,1,1,1,1,1,1,2,1,0,1,0,0,0,2,1343,1
231389,83,1,0,1,1,1,1,1,1,1,2,1,3,1,3,0,12,2,727,1


Get rid of data where the target variable is NA

In [4]:
final_df = model_df[~model_df['confidence_score'].isna()]

Split the data into train and test

In [6]:
from sklearn.model_selection import train_test_split



X = final_df.drop(columns = ['confidence_score'])
y = final_df['confidence_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Create the linear model

In [7]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def fit_model(X_train, X_test, y_train, y_test, model):
    '''
    Fits the model to the data and returns the model metrics
    '''
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train=model.predict(X_train)

    # Evaluate the test set RMSE
    rmse_test = mean_squared_error(y_test, y_pred)**(1/2)
    rmse_train = mean_squared_error(y_train, y_pred_train)**(1/2)

    # Compute the array containing the 5-folds CV MSEs
    MSE_CV_scores = - cross_val_score(model, X_test, y_test, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    # Compute the 5-folds CV RMSE
    RMSE_CV = (MSE_CV_scores.mean())**(1/2)

    return {'y_test': y_test, 'y_pred': y_pred, 'test_rmse': rmse_test, 'train_rmse': rmse_train, 'cv': RMSE_CV, 'model': model}

Results are good. Not over or underfit.

In [9]:
lm_results = fit_model(X_train, X_test, y_train, y_test, lm)
print(f"Test RMSE: {round(lm_results['test_rmse'], 3)}")
print(f"Train RMSE: {round(lm_results['train_rmse'], 3)}")
print(f"Cross Validated RMSE: {round(lm_results['cv'], 3)}")

Test RMSE: 12.492
Train RMSE: 12.471
Cross Validated RMSE: 12.495


In [10]:
final_model = lm_results['model']

In [11]:
weights = final_model.coef_
intercept = final_model.intercept_

In [12]:
feature_weights = dict(zip(X_train.columns, weights))
weights_df = pd.DataFrame([feature_weights])

In [13]:
weights_df

,website_url_binary,twitter_binary,linkedin_binary,location_identifiers_value_binary,phone_number_binary,short_description_binary,legal_name_binary,identifier_permalink_binary,contact_email_binary,status_encoded,operating_status_encoded,revenue_range_encoded,rank_band,num_employees_encoded,ipo_status_encoded,years_since_founded_band,company_type_encoded,days_since_updated,has_category_groups
0,1.061411,0.295338,0.298461,-1.509903e-14,4.34306,-4.529710e-14,2.664535e-14,-2.242651e-14,-0.83669,-0.678093,87.186859,0.51724,-0.2015,0.586561,-0.598856,0.210626,-0.029423,0.001795,1.089707


# Use the model on the data

In [17]:
query = """
SELECT *
FROM `sandbox-b1.safe_prod_exploration.crunchbase_final`

"""
query_job = client.query(query)

labeled_df = query_job.to_dataframe()


In [18]:
to_predict = labeled_df.drop(columns = ['name', 'category', 'country', 'years_since_founded'])

In [19]:
predictions = final_model.predict(to_predict[to_predict.columns])
labeled_df["model_output"] = [round(x) for x in predictions]

Clip due to impossible scores

In [20]:
labeled_df['model_output'] = labeled_df['model_output'].clip(lower=0, upper=100)

# Gut check some companies

In [21]:
labeled_df[labeled_df['name'] == 'Amazon']

,name,category,country,website_url_binary,twitter_binary,linkedin_binary,location_identifiers_value_binary,phone_number_binary,short_description_binary,legal_name_binary,...,revenue_range_encoded,rank_band,num_employees_encoded,ipo_status_encoded,years_since_founded,years_since_founded_band,company_type_encoded,days_since_updated,has_category_groups,model_output
3516771,Amazon,Administrative Services,United States,1,1,1,1,1,1,1,...,8,4,9,2,31,12,2,314,1,93


In [22]:
labeled_df[labeled_df['name'] == 'Google']

,name,category,country,website_url_binary,twitter_binary,linkedin_binary,location_identifiers_value_binary,phone_number_binary,short_description_binary,legal_name_binary,...,revenue_range_encoded,rank_band,num_employees_encoded,ipo_status_encoded,years_since_founded,years_since_founded_band,company_type_encoded,days_since_updated,has_category_groups,model_output
188095,Google,Apps,United States,1,1,1,1,0,1,1,...,8,5,9,2,27,12,2,315,1,88


In [23]:
labeled_df[labeled_df['name'] == 'NVIDIA']

,name,category,country,website_url_binary,twitter_binary,linkedin_binary,location_identifiers_value_binary,phone_number_binary,short_description_binary,legal_name_binary,...,revenue_range_encoded,rank_band,num_employees_encoded,ipo_status_encoded,years_since_founded,years_since_founded_band,company_type_encoded,days_since_updated,has_category_groups,model_output
2354218,NVIDIA,Artificial Intelligence (AI),United States,1,1,1,1,1,1,1,...,8,5,9,2,32,12,2,305,1,92


Rank countries, accounting for the amount of entries each country has.

Do not pay attention to the score itself, pay attention to the rank

In [24]:
total_score = labeled_df.groupby('country')['model_output'].sum()
total_count = labeled_df.groupby('country')['model_output'].count()

weighted_scores = (total_score / total_count) * (total_count / total_count.max())

# Sort
top_countries = weighted_scores.sort_values(ascending=False)

In [25]:
top_countries

,model_output
country,
United States,83.052121
United Kingdom,14.138538
India,10.468129
Germany,7.369582
Canada,6.690260
...,...
"Federated States of""",0.000111
Bouvet Island,0.000109
Tuvalu,0.000058


Top Industries

In [26]:
top_industries = labeled_df.groupby('category')['model_output'].mean().sort_values(ascending=False)

In [27]:
top_industries

,model_output
category,
Manufacturing,86.565592
Government and Military,86.145336
Design,85.924990
Administrative Services,85.590392
Consumer Goods,85.531929
Transportation,85.299986
Real Estate,85.290284
Other,85.214922
Platforms,85.174359
